## 1. Tools
-  LangChain 노트 CH16 참고

### 1) 필요한 라이브러리 설치하기

In [ ]:
#!uv add langchain langchain-openai dotenv

### 2) 환경변수 읽어오기

In [3]:
import dotenv
dotenv.load_dotenv(override=True)

True

### 3) Built-In 도구: TavilySearchResults

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

# 도구 생성
tool = TavilySearchResults(
    max_results=6,
    include_answer=True,
    include_raw_content=True,
    include_domains=["github.io", "wikidocs.net"]
)

# 도구 사용
result = tool.invoke("대한민국의 수도는 어디인가요?")

print(result)

[{'title': '[2024년 UPDATE] OpenAI Python API 튜토리얼 - 채팅(chat) ...', 'url': 'https://teddylee777.github.io/openai/openai-api-tutorial-01/', 'content': '대한민국의 수도는 서울입니다. # 첫 번째 질문 ask("대한민국의 수도는 어디인가요?") \'대한민국의 수도는 서울입니다. \'', 'score': 0.90907925}, {'title': '10. GPT4ALL - <랭체인LangChain 노트>', 'url': 'https://wikidocs.net/233806', 'content': '대한민국(South Korea)의 수도는 서울입니다. 서울은 약 1000만 명의 인구를 가진 대도시로, 한반도 북부에 위치해 있습니다. 세계에서 가장 큰 도시 중 하나로 간주되며', 'score': 0.902481}, {'title': '[논문리뷰]RAG: Retrieval-Augmented Generation for ...', 'url': 'https://meaningful96.github.io/nr/rag/', 'content': 'Wikidata에 검색된 passage: 대한민국의 수도는 서울특별시이며 한강이 도시를 관통한다. 이처럼 질문과 검색된 외부데이터(passage)어디에도 거짓', 'score': 0.8939131}, {'title': '04. OpenAI API 사용(GPT-4o 멀티모달)', 'url': 'https://wikidocs.net/233343', 'content': "'대한민국의 수도는 서울입니다. 서울은 대한민국의 정치, 경제, 문화의 중심지로서 많은 인구와 다양한 명소를 자랑하는 도시입니다.' Copy response", 'score': 0.88372874}, {'title': '02. 퓨샷 프롬프트(FewShotPromptTemplate) - <랭체인 ...', 'url': 'https://wikidocs.net/233348

### 4) 사용자 정의 도구 (Custom Tool)
- @tool 데코레이터 이용하기

In [5]:
from langchain.tools import tool

# 데코레이터를 사용하여 함수를 도구로 변환
@tool 
def add_numbers(a: int, b: int) -> int:
    """두 개의 숫자를 더합니다."""
    return a + b

@tool
def multiply_numbers(a: int, b: int) -> int:
    """두 개의 숫자를 곱합니다."""
    return a * b


In [7]:
# 도구 사용
result = add_numbers.invoke({"a": 1, "b": 2})

print(result)

result = multiply_numbers.invoke({"a": 3, "b": 4})
print(result)

3
12


## 2. Binding Tools

### 1) LLM에 바인딩할 Tool정의
- 도구를 정의할 때 @tool 데코레이터를 사용하여 도구를 정의합니다
- docstring 은 가급적 영어로 작성하는 것을 권장합니다.

In [ ]:
#!uv add beautifulsoup4

In [9]:
import re
import requests
from bs4 import BeautifulSoup
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

@tool
def add_function(a: float, b: float) -> float:
    """Add two numbers together."""
    return a + b

@tool
def naver_news_crawl(news_url: str) -> str:
    """Crawl the naver news from the given URL and return the content."""
    response = requests.get(news_url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        
        title = soup.find("h2", id="title_area").get_text()
        content = soup.find("div", id="content").get_text()
        cleaned_title = re.sub(r"\n{2,}", "\n", title)
        cleaned_content = re.sub(r"\n{2,}", "\n", content)
        
        return cleaned_title + "\n" + cleaned_content
    else:
        return f"Failed to crawl the news from the given URL: {news_url}"
    
tools = [get_word_length, add_function, naver_news_crawl]

### 2) bind_tool()로 LLM에 도구 바인딩

In [13]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

llm_with_tools = llm.bind_tools(tools)

llm_with_tools.invoke("대한민국 단어 길이는 어떻게 되나요?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ePntMYt60NSpDdCnaBOitmFX', 'function': {'arguments': '{"word":"대한민국"}', 'name': 'get_word_length'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 110, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'id': 'chatcmpl-BRxICbzGTIPcoEmy3zOI6GQPR9YSi', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-db32e33c-8ca4-4647-82fb-b54dbdd562c6-0', tool_calls=[{'name': 'get_word_length', 'args': {'word': '대한민국'}, 'id': 'call_ePntMYt60NSpDdCnaBOitmFX', 'type': 'tool_call'}], usage_metadata={'input_tokens': 110, 'output_tokens': 17, 'total_tokens': 127, 'input_token_details': {'audio': 0, 'cache_read':

In [15]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

chain = llm_with_tools | JsonOutputToolsParser(tools=tools)

tool_call_results = chain.invoke("대한민국 단어 길이는 어떻게 되나요?")

tool_call_results


[{'args': {'word': '대한민국'}, 'type': 'get_word_length'}]

In [17]:
# tool을 찾아 args를 전달하여 도구 실행하는 함수
def execute_tool(tool_call_results):
    for tool_call_result in tool_call_results:
        # 도구의 이름과 인자를 추출
        tool_name = tool_call_result["type"]
        tool_args = tool_call_result["args"]
        
        matching_tool = next((tool for tool in tools if tool.name == tool_name), None)
        
        if matching_tool:
            # 도구 실행
            tool_result = matching_tool.invoke(tool_args)
            print(f"Tool {tool_name} called with args: {tool_args}")
            print(f"Tool {tool_name} returned: {tool_result}")
        else:
            print(f"Tool {tool_name} not found")
        
        
execute_tool(tool_call_results)

Tool get_word_length called with args: {'word': '대한민국'}
Tool get_word_length returned: 4


In [18]:
chain = llm_with_tools | JsonOutputToolsParser(tools=tools) | execute_tool

chain.invoke("대한민국 단어 길이는 어떻게 되나요?")



Tool get_word_length called with args: {'word': '대한민국'}
Tool get_word_length returned: 4


### 3) Agent에 도구 바인딩
- agent_scratchpad : 에이전트가 임시로 저장하는 변수
- AgentExecutor: 실제로 llm 호출, 올바른 도구로 라우팅, 실행, 모델 재호출 등을 위한 실행 루프를 생성

In [20]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent

# Agent 프롬프트 생성
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can use tools to answer questions."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    max_iterations=10,
    max_execution_time=10,
    handle_parse_errors=True,
)

result = agent_executor.invoke({"input": "대한민국 단어 길이는 어떻게 되나요?"})

print(result)




> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': '대한민국'}`


4"대한민국"이라는 단어의 길이는 4자입니다.

> Finished chain.
{'input': '대한민국 단어 길이는 어떻게 되나요?', 'output': '"대한민국"이라는 단어의 길이는 4자입니다.'}
